# Setup Beta Account

We need to setup beta account IAM role, and upload training dataset into s3 bucket.

* Login AWS account: mlops-beta
* user: mlops-beta-admin

## IAM SageMakerExecutionRole

* Role Name: SageMakerExecutionRole
* attach policy `AmazonSageMakerFullAccess`
    ![](./img/09.png)

* Add inline policy `beta-sagemake-policy`

In [ ]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "VisualEditor0",
            "Effect": "Allow",
            "Action": [
                "ecr:GetDownloadUrlForLayer",
                "ecr:BatchGetImage",
                "ecr:BatchCheckLayerAvailability"
            ],
            "Resource": "arn:aws:ecr:*:<mlops-devops-accountid>:repository/*"
        }
    ]
}

## S3 Bucket -- This can skip if you don't use s3 cp

* Bucket name: `<your_id>-beta-sagemaker-repo-us-west-2` , S3 Bucket name should be unique.
* region: US West (Oregon)
* folder: iris/input
* Download the iris sample file [here](https://raw.githubusercontent.com/awslabs/amazon-sagemaker-examples/master/advanced_functionality/scikit_bring_your_own/data/iris.csv) and upload the file into s3://beta-sagemaker-repo-us-west-2/iris/input
* Setup the bucket policy. Need to replace the `<mlops-devops-accountid>`

In [ ]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "AWS": "arn:aws:iam::<mlops-devops-accountid>:role/SageMakerExecutionRole"
            },
            "Action": "s3:GetObject",
            "Resource": "arn:aws:s3:::<your_id>-beta-sagemaker-repo-us-west-2/*"
        }
    ]
}

# Setup Dev Account

## Prepare trigger beta training code

Login AWS console:

* account: mlops-dev
* user: mlops-dev-developers

Go to cloud9: `my-mlops-works`

* `mkdir trigger-beta-training`
* `aws --profile mlops-devops-developers codecommit create-repository --repository-name trigger-beta-training`
* `cd trigger-beta-training`
* `git init`
* `git remote add origin https://git-codecommit.us-west-2.amazonaws.com/v1/repos/trigger-beta-training`
* `touch trigger-beta-training.py`

And paste the following code into trigger-beta-training.py. Then commit the code into repo.

**Important: SageMaker Job Name don't allow under_score**



In [ ]:
import time, boto3, sys
import sagemaker as sage 
 
# Create unique job name 
job_name_prefix = 'scikit-bring-your-own' 
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime()) 
job_name = job_name_prefix + timestamp 
 
beta_access_key = sys.argv[1]
beta_secrete_key = sys.argv[2]

#devops
region_devops = boto3.Session().region_name
account_devops = boto3.client('sts').get_caller_identity()['Account']

# beta
ss_beta = boto3.Session(aws_access_key_id=beta_access_key, aws_secret_access_key=beta_secrete_key) 
sm_beta = sage.Session(boto_session=ss_beta) 
account_beta = ss_beta.client('sts').get_caller_identity()['Account'] 

image = '{}.dkr.ecr.{}.amazonaws.com/scikit_bring_your_own:latest'.format(account_devops, region_devops) 
role = 'arn:aws:iam::{}:role/SageMakerExecutionRole'.format(account_beta) 
data_location = 's3://<your_id>-beta-sagemaker-repo-us-west-2/iris/input/iris.csv'
output_location = 's3://<your_id>-beta-sagemaker-repo-us-west-2/iris/output' 

tree = sage.estimator.Estimator(image, 
    role, 1, 'ml.c4.2xlarge', 
    output_path=output_location, 
    sagemaker_session=sm_beta) 
tree.fit(inputs=data_location, job_name=job_name, wait=False)

print('job created at beta account: ' + job_name)


# Setup devops account

Login AWS console:
* account: mlops-devops
* user: mlops-devops-admin
* region: Oregon

## Config AWS Secrets Manager

Click on store a new secret
* secret name: beta/sagemaker
* secret type: other type
* key: beta-sagemaker-access, beta-sagemaker-secret
    ![](./img/10.png)


## Codebuild

* project name: `beta-training-scikit-bring-your-own`
* description: `trigger training scikit-bring-your-own in beta account`
* repoitory: `trigger-beta-training`
* runtime image: aws/codebuild/amazonlinux2-x86_64-standard:2.0
* new service role: `codebuild-beta-training--service-role`
* buildspec.yaml as the following


In [ ]:
version: 0.2

env:
  secrets-manager:
     beta_access_key: beta/sagemaker:beta-sagemaker-access
     beta_secret_key: beta/sagemaker:beta-sagemaker-secret
phases:
  install:
    runtime-versions:
      python: 3.8
  build:
    commands:
      - pip install sagemaker
      - python trigger-beta-training.py $beta_access_key $beta_secret_key
        

## Fixed CodeBuild service role policy

Click on the codebuild project to start build. You will get the failure about `secretsmanager:GetSecretValue`

In the IAM, add inline policy for the service role: `codebuild-beta-training--service-role`.
* policy name: codebuild-trigger-beta-training
* policy json:

In [ ]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "VisualEditor0",
            "Effect": "Allow",
            "Action": "secretsmanager:GetSecretValue",
            "Resource": "arn:aws:secretsmanager:*:*:secret:beta/sagemaker-*"
        }
    ]
}

## Share ECR repository

* In the ECR repsoity: `scikit_bring_your_own`, add the Permissions with following policy.
* Don't forget to replace the `<mlops-beta-account-id>` and `<mlops-prod-account-id>` with correct account id number.
* Policy Generator: https://awspolicygen.s3.amazonaws.com/policygen.html 


In [ ]:
{
  "Version": "2008-10-17",
  "Statement": [
    {
      "Sid": "AllowPull",
      "Effect": "Allow",
      "Principal": {
        "AWS": "arn:aws:iam::<mlops-beta-account-id>:role/SageMakerExecutionRole"
      },
      "Action": [
        "ecr:BatchCheckLayerAvailability",
        "ecr:BatchGetImage",
        "ecr:GetDownloadUrlForLayer"
      ]
    },
    {
      "Sid": "AllowPull",
      "Effect": "Allow",
      "Principal": {
        "AWS": "arn:aws:iam::<mlops-prod-account-id>:role/SageMakerExecutionRole"
      },
      "Action": [
        "ecr:BatchCheckLayerAvailability",
        "ecr:BatchGetImage",
        "ecr:GetDownloadUrlForLayer"
      ]
    }
  ]
}

You will see the following policy in ECR
![](./img/11.png)

You will see the sucess Information in the codebuild output and job name

In [ ]:
Successfully installed boto3-1.10.46 botocore-1.13.46 idna-2.7 numpy-1.18.0 protobuf-3.11.2 protobuf3-to-dict-0.1.5 requests-2.20.1 sagemaker-1.49.0 scipy-1.4.1 smdebug-rulesconfig-0.1.2 urllib3-1.24.3 
WARNING: You are using pip version 19.1.1, however version 19.3.1 is available. 
You should consider upgrading via the 'pip install --upgrade pip' command. 
 
[Container] 2020/01/06 12:47:22 Running command python trigger-beta-training.py $beta_access_key $beta_secret_key $mlops_account_id 
WARNING:root:pandas failed to import. Analytics features will be impaired or broken. 
job created at beta account: scikit-bring-your-own-2020-01-06-12-47-23 
 
[Container] 2020/01/06 12:47:24 Phase complete: BUILD State: SUCCEEDED 
[Container] 2020/01/06 12:47:24 Phase context status code:  Message:  
[Container] 2020/01/06 12:47:24 Entering phase POST_BUILD 
[Container] 2020/01/06 12:47:24 Phase complete: POST_BUILD State: SUCCEEDED 
[Container] 2020/01/06 12:47:24 Phase context status code:  Message:  

In mlops-beta account, you will see SageMaker training job:
![](./img/12.png)

# Congradulations

You done the following redbox in the architecture diagram
![](./img/13.png)